# Setup

In [ ]:
%%capture
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y
!apt-get install python-numpy python-scipy -y
!pip install rasterio
!pip install fiona
!pip install geopandas
!pip install PyCRS
!pip install -i https://test.pypi.org/simple/ gis-utils-pkg-dillhicks==0.0.76


import os
from tqdm.autonotebook import tqdm
import numpy as np
import gis_utils.raster as raster
from rasterio.plot import reshape_as_image
import rasterio
from rasterio.plot import reshape_as_raster, reshape_as_image

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

E: Package 'python-software-properties' has no installation candidate


In [ ]:
!wget https://raw.githubusercontent.com/bnsreenu/python_for_microscopists/master/229_smooth_predictions_by_blending_patches/smooth_tiled_predictions.py

# Loading Data


In [ ]:
!cp /content/drive/MyDrive/srcnn/seg_tiles.zip .
!unzip seg_tiles.zip 

In [ ]:
from cv2 import resize
import cv2


def load_data(input_dir):
    lr_image_filenames = [os.path.join(input_dir, x) for x in os.listdir(input_dir) if x.endswith(".tif") and x.startswith("S2_")]

    hr_image_filenames = [os.path.join(input_dir,"hr_" + os.path.split(x)[1]) for x in lr_image_filenames]
    #Loading LR tiles (getting filenames with "hr_" removed)
    label_filenames = [os.path.join(input_dir,"label_hr_" + os.path.split(x)[1]) for x in lr_image_filenames]

    print(label_filenames)
    labels = []
    hr_images = []
    lr_images = []

    label_meta = []
    hr_image_meta = []
    lr_image_meta = []

    transforms = []
    for index in tqdm(range(len(lr_image_filenames))):
        

        label, label_meta_cur = raster.load_image(label_filenames[index])
        lr_image, lr_image_meta_cur = raster.load_image(lr_image_filenames[index])
        hr_image, hr_image_meta_cur = raster.load_image(hr_image_filenames[index])
                
        labels.append(np.squeeze(reshape_as_image(label.read())))
        lr_images.append(resize(reshape_as_image(lr_image.read()), (48,48), interpolation = cv2.INTER_AREA))
        hr_images.append(np.squeeze(reshape_as_image(hr_image.read())))
        
        
        label_meta.append(label_meta_cur)
        lr_image_meta.append(lr_image_meta_cur)
        hr_image_meta.append(hr_image_meta_cur)
        transforms.append(hr_image.transform)

    return labels, lr_images, hr_images, label_meta, lr_image_meta, hr_image_meta, lr_image_filenames, hr_image_filenames, label_filenames, transforms

In [ ]:
labels_train, lr_images_train, hr_images_train,\
labels_train_meta, lr_images_train_meta, hr_images_train_meta,\
lr_images_train_filenames, hr_images_train_filenames,\
labels_train_filenames, hr_train_transforms = load_data("train")

labels_test, lr_images_test, hr_images_test,\
labels_test_meta, lr_images_test_meta, hr_images_test_meta,\
lr_images_test_filenames, hr_images_test_filenames,\
labels_test_filenames, hr_test_transforms = load_data("test")

labels_train = np.asarray(labels_train)
lr_images_train = np.asarray(lr_images_train)
hr_images_train = np.asarray(hr_images_train)

labels_test = np.asarray(labels_test)
lr_images_test = np.asarray(lr_images_test)
hr_images_test = np.asarray(hr_images_test)


# Loading SRCNN + Generating Data

In [ ]:
!cp /content/drive/MyDrive/srcnn/sr_checkpoint.h5 .

from tensorflow.keras.layers import Activation
from tensorflow.keras import Sequential
import tensorflow as tf
from tensorflow.keras import Model, Input, regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D, Add, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K

def PSNRLoss(y_true, y_pred):

    max_pixel = 1.0
    return (10.0 * K.log((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true), axis=-1)))) / 2.303


srcnn = Sequential()
srcnn.add(Conv2D(64,9,padding='same',input_shape=(48,48,4)))
srcnn.add(Activation('relu'))
srcnn.add(Conv2D(32,1,padding='same'))
srcnn.add(Activation('relu'))
srcnn.add(Conv2D(4,5,padding='same'))
srcnn.compile(optimizer='adam', loss='mean_squared_error', metrics=[PSNRLoss])

In [ ]:
srcnn.load_weights('sr_checkpoint.h5')

In [ ]:
import os
from smooth_tiled_predictions import predict_img_with_smooth_windowing


In [ ]:
r_train = srcnn.predict(lr_images_train)
r_test = srcnn.predict(lr_images_test)

# DeeplabV3


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=tf.keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

In [ ]:
import tensorflow as tf

def DeeplabV3Plus(image_size, num_classes):
    model_input = tf.keras.Input(shape=(image_size, image_size, 4))
    resnet50 = tf.keras.applications.ResNet50(
        weights=None, include_top=False, input_tensor=model_input
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return tf.keras.Model(inputs=model_input, outputs=model_output)




In [ ]:
hr_images_train.shape

In [ ]:
from cv2 import resize
import cv2

dlab_hr = np.array([resize(image, (240,240), interpolation = cv2.INTER_AREA) for image in hr_images_train])
dlab_hr_test = np.array([resize(image, (240,240), interpolation = cv2.INTER_AREA) for image in hr_images_test])
dlab_labels = np.array([resize(image, (240,240), interpolation = cv2.INTER_AREA) for image in labels_train])
dlab_labels_test = np.array([resize(image, (240,240), interpolation = cv2.INTER_AREA) for image in labels_test])

dlab_hr.shape

In [ ]:
num_classes = 2
tf.keras.backend.clear_session()

sr_model = DeeplabV3Plus(image_size=240, num_classes=num_classes)


sr_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics = ['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


early_stopper = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=50, verbose=1, mode='min')
model_checkpoint =  ModelCheckpoint('unet_checkpoint_aug.h5', save_best_only = True)

history = sr_model.fit(dlab_hr, dlab_labels,
            epochs=50,
            validation_data=(dlab_hr_test, dlab_labels_test),
            callbacks=[early_stopper, model_checkpoint])


# Loading DeepLab


In [ ]:
!cp /content/drive/MyDrive/srcnn/deeplab_checkpoint.h5 .


num_classes = 2

sr_model = DeeplabV3Plus(image_size=240, num_classes=num_classes)
sr_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics = ['accuracy'])
srcnn.load_weights('deeplab_checkpoint.h5')

# Mosaicing Visualizations

In [ ]:
!cp /content/drive/MyDrive/srcnn/tiles.zip .


!unzip tiles.zip -d testing

In [ ]:
from cv2 import resize
import cv2



def load_unresolved(input_dir):
    target_filenames = [os.path.join(input_dir, x) for x in os.listdir(input_dir) if x.endswith(".tif")]
    lr_images = []

    lr_meta = []

    bounds = []
    for index in tqdm(range(len(target_filenames))):
        
        lr_image = rasterio.open(target_filenames[index])
        print(lr_image.read().shape)

                
        lr_images.append(resize(reshape_as_image(lr_image.read()), (48,48), interpolation = cv2.INTER_AREA))
        
        lr_meta.append(lr_image.meta)
        bounds.append(lr_image.bounds)

    return lr_images, lr_meta, target_filenames, bounds

In [ ]:
from rasterio.transform import from_bounds

!cp /content/drive/MyDrive/srcnn/site1.tif .


In [ ]:

multiple = 3

lr_image = rasterio.open("site1.tif")
lr_array = resize(reshape_as_image(lr_image.read()), (lr_image.width * multiple, lr_image.height * multiple), interpolation = cv2.INTER_AREA)



sr_input = predict_img_with_smooth_windowing(
    lr_array,
    window_size=48,
    subdivisions=2,  # Minimal amount of overlap for windowing. Must be an even number.
    nb_classes=4,
    pred_func=(
        lambda img_batch_subdiv: srcnn.predict((img_batch_subdiv))
    )
)

import matplotlib.pyplot as plt
predictions_smooth = np.argmax(predictions_smooth, axis = 2)

plt.imshow(predictions_smooth)
plt.figure(figsize=(20, 20))

In [ ]:
predictions_smooth = predict_img_with_smooth_windowing(
    input,
    window_size=240,
    subdivisions=6,  # Minimal amount of overlap for windowing. Must be an even number.
    nb_classes=2,
    pred_func=(
        lambda img_batch_subdiv: sr_model.predict((img_batch_subdiv))
    )
)

In [ ]:
import matplotlib.pyplot as plt
predictions_smooth = np.argmax(predictions_smooth, axis = 2)

plt.imshow(predictions_smooth)
plt.figure(figsize=(20, 20))